In [3]:
import requests
import pandas as pd
from secrets_config import iex_api_key

In [55]:
#The value of stock is taken data.csv file.
#Figuring out to configure yml file still not applied.
def extract_per_stock(
            iex_api_key:str,
            stock_ticker:str=None,
        )->pd.DataFrame:
        headers = {
            "token" : iex_api_key
        }
        """
        Extracting data from the intraday stock API. 
        - iex_api_key: api key 
        - stock_tickere: code of the stock e.g. AAPL for APPLE stock
        """
        params = {
            "token" : iex_api_key
        }
        base_url = f"https://cloud.iexapis.com/stable/stock/{stock_ticker}/intraday-prices"
        response = requests.get(base_url, params=params)
        if response.status_code == 200: 
            stock_data = response.json()
        else: 
            raise Exception("Extracting weather api data failed. Please check if API limits have been reached.")
        df_stock_codes = pd.json_normalize(stock_data)
        return df_stock_codes



In [54]:

def extract_stocks(
            iex_api_key:str, 
        )->pd.DataFrame:
    response_data = []
     # read list of stock_codes
    df_stock_codes = pd.read_csv("data\stock_codes.csv")
    # request data for each stock_code (json) and push to a list 
    df_concat = pd.DataFrame()
    for exchange_code in df_stock_codes["exchange_code"]:
        df_extracted = extract_per_stock(stock_ticker=exchange_code, iex_api_key= iex_api_key)
        df_extracted["stock_code"] = exchange_code
        df_concat = pd.concat([df_concat,df_extracted])
    return df_concat
    

In [ ]:
def transform(
            df:pd.DataFrame
        )->pd.DataFrame:
        """
        Transform the raw dataframes. 
        - df: the dataframe produced from extract_stocks(). 
        """
        # renaming columns
        df = df.rename(columns={
            "numberOfTrades": "numberoftrades"
            })

        #Adding column datetime that includes date and time both. Conversion of minutes into 24-hours format.
        df["datetime"] = pd.to_datetime(df["minute"])
        #Droping columns "date", "minutes" and labels, added datetime in prior step
        df = df.drop(df.columns[[0,1,2]],axis=1)
        #Difference between open and close value across rows.
        df["difference"] = (df["close"]-df["open"])
        #Creation of new dataframe to analyse the raw dataframe for per day stock vlues.
        df1 = pd.DataFrame()
        #MAX AND MINOF OPEN , CLOSE, HIGH, LOW VALUE GROUPING ON PER STOCK BASIS and APPLYING LAMBDA FUNCTION
        df1["max_open_value_per_day"] = df.groupby('stock_code').apply(lambda df: df["open"].max())
        df1["min_open_value_per_day"] = df.groupby('stock_code').apply(lambda df: df["open"].min())
        df1["max_close_value_per_day"] = df.groupby('stock_code').apply(lambda df: df["close"].max())
        df1["min_close_value_per_day"] = df.groupby('stock_code').apply(lambda df: df["close"].min())
        df1["max_high_per_day"] = df.groupby('stock_code').apply(lambda df: df["high"].max())
        df1["min_high_per_day"] = df.groupby("stock_code").apply(lambda df: df["high"].min())
        df1["max_low_per_day"] = df.groupby('stock_code').apply(lambda df: df["low"].max())
        df1["min_low_per_day"] = df.groupby("stock_code").apply(lambda df: df["low"].min())
        # Difference sum for per stock
        df1["status_difference"] = df.groupby("stock_code").apply(lambda df: df["difference"].sum())

        #MEAN VALUE FOR trades and volume PER DAY PER STOCK THROUGH GROUPING BY STOCK CODE AND LAMBDA FUNCTION APPLYING.
        df1["trades_mean"] = df.groupby("stock_code").apply(lambda df: df["numberoftrades"].mean())
        df1["volume_mean"] = df.groupby("stock_code").apply(lambda df: df["volume"].mean())
        
        return df1